# RADMC-3D Volume Rendering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import astropy.constants as c

import radmc3d_volume_rendering as rvr

au = c.au.cgs.value

## Setup

We need to set the path to the radmc3d source folder `src` when initializing. It will then compile the 'normal' radmc3d together with the userdefined function.

In [ ]:
ren = rvr.Renderer(src_dir='~/CODES/radmc3d-2.0/src/')

Next, we need some data: grid and densities. You can assign them directly:


```python
ren.t_i = theta_ii
ren.p_i = phi_i
ren.r_i = r_i
ren.rho = rho
```

where $\theta$ is defined to start from 0 at the pole and the order of indices of the density is: $\rho(r, \theta, \phi)$.

If those variables are stored under the same names (`r_i, t_i, p_i, rho`) we can load that directly (or a dictionary with the same keywords):

In [ ]:
ren.read_data('fargo3d_data.npz')

And to write out the density for radmc3d, we do:

In [ ]:
ren.write_input()

In [ ]:
ren._close_radmc3d_child()

We write out the desired parameters of a gaussian transfer function:

**WARNING:** right now this is only read-in once, so this needs to be fixed in the image calculation, perhaps defining this as `myaction`? Right now, if you change those parameters, you need to restart the child process with `ren._stop_radmc_child()`

In [ ]:
ren.write_transfer_options(mean=0.5 * ren.rho.max(), sigma=0.1 * ren.rho.max())

## Rendering

Rendering should now work by directly talking to the child process:

In [ ]:
cmd = 'image lambda 5 sizeradian 1.0 projection 1 posang 0 locobsau 1 -0.25 0.25 pointau 0.98 0.19 0 nofluxcons npix 300'

In [ ]:
im = ren.make_image(cmd)

but `ren.callit` and `ren.plotit` should still work for calls to radmc3d without using the child-process method.

In [ ]:
vmax = im.im.max()

f, ax = plt.subplots(dpi=150)
ax.pcolormesh(im.xi, im.yi, im.im.T, norm=LogNorm(vmin=1e-4*vmax, vmax=0.5 * vmax, clip=True), cmap='afmhot')
ax.set_aspect(1)

Sanity check: vertical sum of density to see if the pattern matches

In [ ]:
Ri, PHi = np.meshgrid(ren.r_i / c.au.cgs.value, ren.p_i, indexing='ij')
_Xi = Ri * np.cos(PHi)
_Yi = Ri * np.sin(PHi)

f, ax = plt.subplots()
ax.pcolormesh(_Xi, _Yi, (ren.rho * 0.5 * (ren.r_i[1:]+ ren.r_i[:-1])[:, None, None] * np.diff(ren.t_i)[None, :, None]).sum(1))
ax.set_aspect('equal')

We can also make RADMC-3D write the vtk_grid and data

In [ ]:
ren.callit(command='vtk_grid')

In [ ]:
ren.callit(command='vtk_gas_density')